##### Using Declarative Mixins with Dataclasses

In the section `Composing Mapped Hierarchies with Mixins`, _Declarative Mixin_ classes are introduced. One _requirement of declarative mixins_ is that __certain constructs that can't be easily duplicated must be given as `callables`__, using the `declared_attr` decorator, such as in the example at `Mixing in Relationships`.

In [1]:
from typing import List
from __future__ import annotations
from dataclasses import dataclass ,field

from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import registry, declared_attr, relationship

In [2]:
mapper_registry = registry()

In [3]:
class RefTargetMixin:
    @declared_attr
    def target_id(cls):
        return Column("target_id", ForeignKey("target.id"))
    
    @declared_attr
    def target(cls):
        return relationship("Target")

This form is supported within the _Dataclasses_ `field()` object by using a `lambda` to __indicate the SQLAlchemy construct inside the `field()`__. Using `declared_attr()` to _surround the lambda_ is __optional__. If we wanted to produce our `User` class above where the _ORM fields came from a mixin_ that is itself a `dataclass`, the form would be as follows.

In [4]:
@dataclass
class UserMixin:
    __tablename__ = "user"
    
    __sa_dataclass_metadata_key__ = "sa"
    
    id: int = field(init=False, metadata={"sa": Column(Integer, primary_key=True)})
    
    addresses: List[Address] = field(
        default_factory=list, metadata={"sa": lambda: relationship("Address")},
    )

In [5]:
@dataclass
class AddressMixin:
    __tablename__ = "address"
    
    __sa_dataclass_metadata_key__ = "sa"
    
    id: int = field(init=False, metadata={"sa": Column(Integer, primary_key=True)})
    user_id: int = field(init=False, metadata={"sa": lambda: Column(ForeignKey("user.id"))})
    email_address: str = field(default=None, metadata={"sa": Column(String(50))})

In [6]:
@mapper_registry.mapped
class User(UserMixin):
    pass

In [7]:
@mapper_registry.mapped
class Address(AddressMixin):
    pass